<a href="https://colab.research.google.com/github/runfish5/tiny-tutorials/blob/main/discord-transcription-bot/jupyter-notebooks/Discord_bot_notebook_WhisperX_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color='winered'>Step 2: Turn on bot

In [1]:
#@title Install requirements
%%capture
'''
This is more cumbersome than necessary, but
this way it creates a requirements.txt
which is desirable.
'''

content = """py-cord
python-dotenv"""

with open("requirements.txt", "w") as file:
    file.write(content)

print("requirements.txt has been created.")

!pip install -r ./requirements.txt
!pip install PyNaCl
!pip install nest_asyncio


%cd /content/
!rm -rf /content/tiny-tutorials
!git clone https://github.com/runfish5/tiny-tutorials.git

%cd /content/tiny-tutorials/discord-transcription-bot

In [2]:
#@title import packages
import tempfile,  os, discord, asyncio, nest_asyncio
nest_asyncio.apply()

from pprint import pprint
from dotenv import load_dotenv

In [3]:
#@title Load environment
env_content = """DISCORD_BOT_TOKEN=<insert-secret-access-token>
"""
with open(".env", "w") as env_file:
    env_file.write(env_content)

!pwd
load_dotenv()

/content/tiny-tutorials/discord-transcription-bot


True

In [4]:
#@title Define Helper function

'''Get bot status'''

def get_bot_status():
  if 'bot_task' in globals() and not bot_task.done():
      print("Bot is RUNNING. 🔴 ")
  else:
      print("Bot is NOT (!) running. 🛑")

get_bot_status()

Bot is NOT (!) running. 🛑


In [5]:
#@title Execute main.py
%cd /content/tiny-tutorials/discord-transcription-bot
exec(open("main.py").read())

# Global process flag
running = True

async def keep_alive():
    global running
    while running:
        await asyncio.sleep(1)  # Keep the loop alive without blocking

# Start the bot and the keep-alive task
loop = asyncio.get_event_loop()
bot_task = loop.create_task(run_bot())
keep_alive_task = loop.create_task(keep_alive())

get_bot_status()

/content/tiny-tutorials/discord-transcription-bot
This script is meant to be run from the notebook.
Bot is RUNNING. 🔴 


In [6]:
#@title bot: off-switch

await bot.close()
running = False

get_bot_status()

Bot is RUNNING. 🔴 


In [7]:
#@title 💾 zip it
'''
For downloads we zip it.
(Google Colab doesn't support download and upload of directories.)
'''
from datetime import datetime
# Get current date in yymmdd format
current_date = datetime.now().strftime("%y%m%d")

# Create base zip filename with date prefix
base_zip_filename = f"{current_date}_recordings.zip"

# Check if file exists and add version number if needed
i = 1
zip_filename = base_zip_filename
while os.path.exists(zip_filename):
    zip_filename = f"{current_date}_recordings_{i}.zip"
    i += 1

# Zip the recordings directory with the dated filename
!zip -r {zip_filename} recordings

print(f"{''}Created zip file: {zip_filename}{''}")

  adding: recordings/ (stored 0%)
Created zip file: 250509_recordings.zip


#<font color='winered'>ASR

In this section the recordings are transcribed.

##<font color='magenta'> Step : Setup WhisperX

<font color='red'> RESTART SESSION

In [8]:
%cd /content
asr_model = 'WHISPERX'

/content


In [9]:
#@title Define script Helpers

GREEN = "\033[92m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
RESET = "\033[0m"
MAGENTA = '\033[35m'
RED = '\033[31m'
ORANGE = "\033[38;5;208m"
RESET = '\033[0m'


In [10]:
#@title pip install whisperx Whisper X
%%capture
"""
Takes some time. (> 2 min 30 sec).
Have to install whisperx copy (transcription AI engine).
"""
try:
    import whisperx
except ImportError:
    # !pip install whisperx
    !pip install git+https://github.com/m-bain/whisperx.git --upgrade
    import whisperx

In [11]:
#@title set your parameters Whisper X
# Whisper is on large-v2 by default
model_size = 'tiny' #@param ["tiny", "small", "medium", "large-v2", "tiny.en", "small.en", "medium.en"]
device = 'cpu' #@param ["cpu", "cuda"]
audio_file_language = 'en' #@param ["en"]

<font color='red'>I haven't figured out how to run WhisperX on google colab GPUs. I'm sorry.

In [12]:
!pwd
%cd /content/tiny-tutorials/discord-transcription-bot

/content
/content/tiny-tutorials/discord-transcription-bot


In [13]:
#@title set the AI model Whisper X


import whisperx
import gc
import model_config
import torch


batch_size = 16 # reduce if low on GPU mem
if device == 'cpu':
  compute_type = 'auto'
else:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)


# 1. Transcribe with original whisper (batched)
model_config.model = whisperx.load_model(model_size, device, compute_type=compute_type)

model_a, metadata_model_a = whisperx.load_align_model(language_code=audio_file_language,#result["language"],
                                              device=device)

print(metadata_model_a)
print("\nModel assigned to configuration")


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 269MB/s]


{'language': 'en', 'dictionary': {'-': 0, '|': 1, 'e': 2, 't': 3, 'a': 4, 'o': 5, 'n': 6, 'i': 7, 'h': 8, 's': 9, 'r': 10, 'd': 11, 'l': 12, 'u': 13, 'm': 14, 'w': 15, 'c': 16, 'f': 17, 'g': 18, 'y': 19, 'p': 20, 'b': 21, 'v': 22, 'k': 23, "'": 24, 'x': 25, 'j': 26, 'q': 27, 'z': 28}, 'type': 'torchaudio'}

Model assigned to configuration


In [14]:
#@title testing
# =============================================================================

# #get Sample from Nvidia:
# !wget -P /content https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# audio_path = '/content/2086-149220-0033.wav'

# audio = whisperx.load_audio(audio_path)
# transcription = model.transcribe(audio, batch_size=batch_size)

# print("Transcription done:")
# print(f"{transcription['segments'][0]['text'][:100]} ...")

##<font color='magenta'>Step 3: Transcribe Audio Files

In [ ]:
!pwd
%cd /content/tiny-tutorials/discord-transcription-bot

/content/tiny-tutorials/discord-transcription-bot
/content/tiny-tutorials/discord-transcription-bot


In [15]:
#@title Download sample audio folder (no need to execute)
'''
not required if you recorded your discord session
'''
# import gdown

# # Replace with your shared link
# shared_link = "https://drive.google.com/file/d/1tEu6xqF9DMQ9FEcXZ4guG3uymxiVEtcZ/view?usp=sharing"
# shared_link = "https://drive.google.com/file/d/1ouoroEO0tbBGXs4LNDCF9vwaM9306jZF/view?usp=sharing"
# # Extract the file ID from the shared link
# file_id = shared_link.split("/")[5]

# # Download the file
# output_path = "/content/downloaded_file.zip"

# gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
# import zipfile

# # Unzip the file
# with zipfile.ZipFile(output_path, 'r') as zip_ref:
#     zip_ref.extractall("/content/extracted_files")


# # ==================================================
# #@title COPY THE RECORDINGS (.wav)

# import os
# import shutil

# # Check if the 'recordings' directory is empty
# recordings_dir = "recordings"
# if not os.listdir(recordings_dir):
#     # Copy the recordings directory from /content/extracted_files/ to current directory
#     shutil.copytree("/content/extracted_files/recordings", recordings_dir, dirs_exist_ok=True)

Downloading...
From: https://drive.google.com/uc?id=1ouoroEO0tbBGXs4LNDCF9vwaM9306jZF
To: /content/downloaded_file.zip
100%|██████████| 1.61M/1.61M [00:00<00:00, 69.2MB/s]


In [23]:
!pwd
%cd /content/tiny-tutorials/discord-transcription-bot

/content/tiny-tutorials/discord-transcription-bot
/content/tiny-tutorials/discord-transcription-bot


In [25]:
#@title transcribe_all_recordings WHISPERX

import os
import json
import time
import model_config  # Assuming this module defines the Whisper model

def transcribe_all_recordings(skip_existing=True):
    """
    Transcribes all audio recordings in the 'recordings' directory using the Whisper model.
    Saves transcriptions as text files and updates metadata.json for each user.
    If skip_existing is True, it will not re-transcribe already existing transcription files.
    """
    model_config.model = model_config.model  # Ensure model is set (redundant here if already set)
    print(f"Model: {model_config.model}")  # Print the model object itself
    print(f"Model Config: {model_config}") # Print the configuration

    recordings_dir = "recordings"
    if not os.path.exists(recordings_dir):
        print("Error: The 'recordings' directory does not exist.")
        return

    for user_id in os.listdir(recordings_dir):
        user_dir = os.path.join(recordings_dir, user_id)
        if not os.path.isdir(user_dir):
            continue

        metadata_path = os.path.join(user_dir, "metadata.json")
        if not os.path.exists(metadata_path):
            print(f"Warning: No metadata.json found for user {user_id}. Skipping.")
            continue

        with open(metadata_path, "r") as f:
            metadata = json.load(f)

        for recording in metadata.get("recordings", []):
            audio_file = recording.get("audio_file")
            if not audio_file:
                print(f"Warning: No audio_file specified in metadata for user {user_id}. Skipping entry.")
                continue

            audio_path = os.path.join(user_dir, audio_file)
            if not os.path.exists(audio_path):
                print(f"Warning: Audio file {audio_path} not found. Skipping.")
                continue

            transcription_file = f"{os.path.splitext(audio_file)[0]}_transcription.txt"
            transcription_path = os.path.join(user_dir, transcription_file)

            if skip_existing and os.path.exists(transcription_path):
                print(f"Skipping existing transcription for {audio_file}.")
                continue

            try:
                start_time = time.time()
                # print_audio_length(audio_path)
                audio = whisperx.load_audio(audio_path)
                transcription = model_config.model.transcribe(audio)
                transcription = whisperx.align(transcription["segments"],
                                               model_a,
                                               metadata_model_a,
                                               audio,
                                               device,
                                               return_char_alignments=False)
                transcription_text = transcription if transcription.get('segments', None) else transcription["text"]
                end_time = time.time()
                duration = end_time - start_time
            except Exception as e:
                print(f"Error: Failed to transcribe {audio_path}: {str(e)}")
                continue

            with open(transcription_path, "w") as f:
                f.write(json.dumps(transcription_text, indent=4, ensure_ascii=False))

            recording["transcription_file"] = transcription_file
            recording["transcription_time_seconds"] = round(duration, 2)
            print(f"Transcribed {audio_file} in {duration:.2f} seconds.")

        with open(metadata_path, "w") as f:
            json.dump(metadata, f, indent=4)

        print(f"Success: Transcriptions completed for user {user_id}")

!pwd
# Example usage:
# transcribe_all_recordings(skip_existing=True)  # Skips existing transcriptions
transcribe_all_recordings(skip_existing=False)  # Re-transcribes all files


/content/tiny-tutorials/discord-transcription-bot
Model: <whisperx.asr.FasterWhisperPipeline object at 0x7ab5535e5810>
Model Config: <module 'model_config' from '/content/tiny-tutorials/discord-transcription-bot/model_config.py'>
Detected language: en (0.81) in first 30s of audio...
Transcribed session_20250509_13M39.wav in 14.95 seconds.
Success: Transcriptions completed for user test_acc_runfish
Detected language: en (0.86) in first 30s of audio...
Transcribed session_20250509_13M39.wav in 17.50 seconds.
Success: Transcriptions completed for user runfishrun


In [26]:
#@title adjust_transcription_times
import json
import os
from datetime import datetime

# Define color codes for output (optional, assuming they’re defined elsewhere if needed)
GREEN = '\033[92m'
RESET = '\033[0m'

def load_json_file(file_path):
    """Loads a JSON file and returns its contents."""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error loading {file_path}: {e}")
        return None

def calculate_time_offset(recording, reference_time):
    """
    Calculates time difference between user start time and the reference time in seconds.

    Args:
        recording (dict): Recording metadata with user_recording_start_time.
        reference_time (datetime): The unified reference time (earliest session_start_time).

    Returns:
        float: Time offset in seconds, or None if parsing fails.
    """
    try:
        user_dt = datetime.strptime(recording["user_recording_start_time"], "%Y-%m-%d %H:%M:%S")
        offset = (user_dt - reference_time).total_seconds()
        print(f"{GREEN}total_seconds offset for {recording['audio_file']}: {offset}{RESET}")
        return offset
    except ValueError as e:
        print(f"Time parsing failed for {recording['audio_file']}: {e}")
        return None

def get_transcription_path(user_dir, audio_file):
    """Constructs and validates the transcription file path."""
    audio_base = os.path.splitext(audio_file)[0]
    trans_file = f"{audio_base}_transcription.txt"
    trans_path = os.path.join(user_dir, trans_file)
    if not os.path.exists(trans_path):
        print(f"Missing transcription: {trans_path}")
        return None
    return trans_path

def adjust_timestamps(transcription, time_offset):
    """Adjusts all start and end times in the transcription by the offset."""
    for segment in transcription["segments"]:
        segment["start"] += time_offset
        segment["end"] += time_offset
        for word in segment["words"]:
            word["start"] = word.get("start", 0) + time_offset
            word["end"] = word.get("end", 0) + time_offset

    for word in transcription["word_segments"]:
        word["start"] = word.get("start", 0) + time_offset
        word["end"] = word.get("end", 0) + time_offset

def save_adjusted_transcription(trans_path, transcription):
    """Saves the adjusted transcription to a new file."""
    new_trans_path = trans_path.replace(".txt", "_adjusted.txt")
    with open(new_trans_path, 'w') as f:
        json.dump(transcription, f, indent=4)
    print(f"Saved adjusted transcription to {new_trans_path}")

def adjust_transcription_times(metadata_path, reference_time):
    """
    Shifts timing in transcription files to align with the earliest session start time.

    Args:
        metadata_path (str): Path to metadata JSON file (e.g., 'recordings/<user_id>/metadata.json').
        reference_time (datetime): The earliest session_start_time across all sessions.
    """
    metadata = load_json_file(metadata_path)
    if not metadata:
        return

    user_dir = os.path.dirname(metadata_path)
    for recording in metadata["recordings"]:
        time_offset = calculate_time_offset(recording, reference_time)
        if time_offset is None:
            continue

        trans_path = get_transcription_path(user_dir, recording["audio_file"])
        if not trans_path:
            continue

        transcription = load_json_file(trans_path)
        if not transcription:
            continue

        adjust_timestamps(transcription, time_offset)
        save_adjusted_transcription(trans_path, transcription)

# Process all user directories
base_dir = "/content/tiny-tutorials/discord-transcription-bot/recordings"
user_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Step 1: Find the earliest session_start_time (T0)
session_start_times = []
for user_dir in user_dirs:
    metadata_path = os.path.join(base_dir, user_dir, "metadata.json")
    metadata = load_json_file(metadata_path)
    if metadata:
        for recording in metadata["recordings"]:
            session_start_times.append(recording["session_start_time"])

if not session_start_times:
    print("No session start times found. Cannot proceed with adjustments.")
else:
    # Convert to datetime objects and find the minimum
    session_start_datetimes = [datetime.strptime(st, "%Y-%m-%d %H:%M:%S") for st in session_start_times]
    T0 = min(session_start_datetimes)
    start_time_str = str(T0)
    print(f"Earliest session start time (T0): {T0}")

    # Step 2: Adjust transcriptions using T0 as the unified reference
    for user_dir in user_dirs:
        metadata_path = os.path.join(base_dir, user_dir, "metadata.json")
        adjust_transcription_times(metadata_path, T0)

Earliest session start time (T0): 2025-05-09 13:51:46
total_seconds offset for session_20250509_13M39.wav: 20.0
Saved adjusted transcription to /content/tiny-tutorials/discord-transcription-bot/recordings/test_acc_runfish/session_20250509_13M39_transcription_adjusted.txt
total_seconds offset for session_20250509_13M39.wav: 7.0
Saved adjusted transcription to /content/tiny-tutorials/discord-transcription-bot/recordings/runfishrun/session_20250509_13M39_transcription_adjusted.txt


In [27]:
# @title create_unified_transcription_object
import os
import json
from datetime import datetime, timedelta

# Custom function to handle datetime serialization
def custom_serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to an ISO 8601 string
    raise TypeError(f"Type {type(obj)} not serializable")

def create_unified_transcription_object( start_time_str, recordings_dir="recordings"):
    """
    Creates a unified JSON-like object from transcription files in the recordings directory.
    Each segment includes the username and is sorted by absolute start time.

    Args:
        recordings_dir (str): Path to the 'recordings' directory.

    Returns:
        list: Sorted list of segment dictionaries with username, start, end, and text.
    """
    all_segments = []

    # Iterate over each user directory in 'recordings'
    for user_dir in os.listdir(recordings_dir):
        user_path = os.path.join(recordings_dir, user_dir)
        if not os.path.isdir(user_path):
            continue

        # Load metadata.json
        metadata_path = os.path.join(user_path, "metadata.json")
        if not os.path.exists(metadata_path):
            print(f"Warning: metadata.json not found in {user_path}")
            continue

        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = json.load(f)

        # Process each recording in metadata
        for recording in metadata.get("recordings", []):
            username = recording.get("username")
            if not username:
                print(f"Warning: No username found in recording {recording}")
                continue

            if not start_time_str:
                print(f"Warning: No user_recording_start_time in {recording}")
                continue
            try:
                recording_start = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
            except ValueError as e:
                print(f"Error parsing time {start_time_str}: {e}")
                continue

            # Derive the adjusted transcription file name
            transcription_file = recording.get("transcription_file")
            if not transcription_file:
                print(f"Warning: No transcription_file in {recording}")
                continue
            # Replace '_transcription.txt' with '_transcription_adjusted.txt'
            adjusted_file = transcription_file.replace("_transcription.txt", "_transcription_adjusted.txt")
            adjusted_path = os.path.join(user_path, adjusted_file)

            # Load the transcription file
            if not os.path.exists(adjusted_path):
                print(f"Warning: {adjusted_file} not found in {user_path}")
                continue

            with open(adjusted_path, 'r', encoding='utf-8') as f:
                transcription = json.load(f)

            # Process each segment
            for segment in transcription.get("segments", []):
                rel_start = segment.get("start")
                rel_end = segment.get("end")
                text = segment.get("text")

                if rel_start is None or rel_end is None or not text:
                    print(f"Warning: Incomplete segment in {adjusted_file}: {segment}")
                    continue

                # Calculate absolute times
                abs_start = recording_start + timedelta(seconds=rel_start)
                abs_end = recording_start + timedelta(seconds=rel_end)

                # Create segment dictionary
                segment_dict = {
                    "username": username,
                    "start": abs_start,  # datetime object
                    "end": abs_end,      # datetime object
                    "text": text
                }
                all_segments.append(segment_dict)

    # Sort all segments by absolute start time
    all_segments.sort(key=lambda x: x["start"])

    return all_segments


unified_object = create_unified_transcription_object(start_time_str)
unified_obj_path = '/content/tiny-tutorials/discord-transcription-bot/recordings/unified_object.json'
final_transcript_path = '/content/tiny-tutorials/discord-transcription-bot/recordings/transcript.txt'
# Save JSON with datetime converted to string
with open(unified_obj_path, 'w') as f:
    json.dump(unified_object, f, indent=2, default=custom_serializer)

In [28]:
#@title transform_transcript_format
from datetime import datetime

def transform_to_transcript(data):
    # List to store groups of consecutive entries
    groups = []
    current_group = None

    # Step 1: Group consecutive entries by speaker
    for entry in data:
        if current_group is None or entry['username'] == current_group['speaker']:
            # If no current group exists, start a new one
            if current_group is None:
                current_group = {
                    'speaker': entry['username'],
                    'start_time': entry['start'],
                    'texts': [entry['text'].strip()]
                }
            # If speaker is the same, append text to current group
            else:
                current_group['texts'].append(entry['text'].strip())
        else:
            # Speaker changed, save current group and start a new one
            groups.append(current_group)
            current_group = {
                'speaker': entry['username'],
                'start_time': entry['start'],
                'texts': [entry['text'].strip()]
            }

    # Don’t forget to append the last group
    if current_group is not None:
        groups.append(current_group)

    # Step 2: Format each group into transcript lines
    transcript_lines = []
    for i, group in enumerate(groups, 1):  # Start numbering at 1
        # Convert the group's start time to HH:MM:SS format
        start_time = datetime.fromisoformat(group['start_time']).strftime("%H:%M:%S")
        # Format the entry: number, [user @ time], and all texts joined with spaces
        entry_text = f"{i} [{group['speaker']} @ {start_time}]\n{' '.join(group['texts'])}\n\n"
        transcript_lines.append(entry_text)

    # Step 3: Join all lines with newlines
    return "\n".join(transcript_lines)

# Example usage with the rest of your code
# Load JSON back into a Python object
with open(unified_obj_path, 'r') as f:
    unified_object = json.load(f)

# Generate transcript
transcript = transform_to_transcript(unified_object)

# Save transcript to file
with open(final_transcript_path, 'w') as f:
    f.write(transcript)

# Print for verification
print(f"{MAGENTA}\nTransformed transcript saved as 'transcript.txt':{RESET}")
print(transcript)


Transformed transcript saved as 'transcript.txt':
1 [runfishrun @ 13:51:53]
Hello, this first account speaking, first round, first engagement.


2 [test_acc_runfish @ 13:52:06]
Hello, this is the second account.


3 [runfishrun @ 13:52:08]
No, this is my second turn again. Awesome. Second term.


4 [test_acc_runfish @ 13:52:23]
Second term, second account.


5 [runfishrun @ 13:52:24]
Second term.


6 [test_acc_runfish @ 13:52:26]
Finish testing now.


7 [runfishrun @ 13:52:27]
Second term.




The smallest model (WhisperX-tiny) picks up the echo because the microphone and speaker for both accounts are in the same room. However, the medium model is able to filter out this issue.

In [ ]:
'''
For downloaed we zip it.
(Google Colab doesnt support download and upload of directories.)
'''
!zip -r recordings.zip recordings

In [ ]:
# !unzip recordings.zip -d recordings_unzip


##<font color='magenta'>Further Information

- comparison of some models (e.g. whisper, whisperX, etc.) https://github.com/SYSTRAN/faster-whisper

- Another Notebook with utilizing: [WhisperX.ipynb](https://colab.research.google.com/github/cnbeining/Whisper_Notebook/blob/master/WhisperX.ipynb#scrollTo=VuO7n7Xa7myj)
    
    - GPU setting
    - WhisperX
    - single audio file
    - translation to 31 languages, fileformat: SRT (SubRip SubTitle)
```
1
00:00:05,000 --> 00:00:10,000
Hello, welcome to our video!
```
```
2
00:00:12,000 --> 00:00:15,000
Today, we will talk about translation.
```

- Another Notebook with utilizing: [WhisperX_Speaker_Diarization.ipynb](https://colab.research.google.com/drive/1IHum-j2AOjVOs_ZoqJ5yBUjf1kI4SLmt?usp=sharing)
    - [Youtube-Video: *Multi Speaker Transcription with Speaker IDs with Local Whisper*](https://www.youtube.com/watch?v=SAIsk0i7KgU)
    - `load_align_model` to get word-level timestamps (like I did)
    - Diarization (multiple persons in one audio file)
        - `diarize_model(audio, min_speakers=2, max_speakers=2)` #or auto-detect


### Diarization
- [pyannote.audio speaker diarization toolkit](https://github.com/pyannote/pyannote-audio)

### real-time streaming automatic speech recognition (ASR).
- https://www.reddit.com/r/Python/comments/170iwzc/i_developed_a_realtime_speech_to_text_library/
    - https://www.youtube.com/watch?v=BKOMqKp436Q
    - https://github.com/KoljaB/RealtimeSTT
- https://www.youtube.com/watch?v=_spinzpEeFM
- https://www.youtube.com/watch?v=k6nIxWGdrS4!
- https://www.youtube.com/watch?v=VpB6bxh4deM
